<a href="https://colab.research.google.com/github/InChil2/cakd3/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [4]:
!pip install --target=$my_path "jpype1<1.1"

     |████████████████████████████████| 3.8 MB 7.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.7/shutil.py", line 566, in move
    os.rename(src, real_dst)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/pip-target-zscqr5hz/lib/python/JPype1-1.0.2.dist-info' -> '/content/notebooks/JPype1-1.0.2.dist-info'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
   

In [11]:
!pip install --target=$my_path konlpy

  Using cached konlpy-0.5.2-py2.py3-none-any.whl (19.4 MB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 15.7 MB 144 kB/s 
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
     |████████████████████████████████| 6.3 MB 48.6 MB/s 
  Using cached JPype1-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (448 kB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
     |████████████████████████████████| 62 kB 610 kB/s 
     |████████████████████████████████| 146 kB 71.9 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 138 kB 76.4 MB/s 
     |████████████████████████████████| 145 kB 78.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatib

In [5]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_train.txt',sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [8]:
import re

train_df = train_df.fillna(' ')

train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))

test_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_test.txt',sep='\t')
test_df= test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))

train_df.drop('id',axis=1, inplace=True)
test_df.drop('id',axis=1, inplace=True)

In [12]:
from konlpy.tag import Okt

okt = Okt()
def ok_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=ok_tokenizer,ngram_range=(1,2),min_df=3,max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
[과제]
GridSearchCV를 이용하여 모델리 및 평가를 아래와 같이 수행하세요/
  - 로지스틱 리그레션,'C':[1,3.5,4.5,5.5,10]
  - cv=3, scoring='accuracy',verbose=1

In [ ]:
lg_clf = LogisticRegression(random_state=0)
params = {'C':[1,3.5,4.5,5.5,10]}
gridcv = GridSearchCV(lg_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
gridcv.fit(tfidf_matrix_train,train_df['label'])
print(grid_cv.best_params_,round(grid_cv.best_score_,4))